In [12]:
import andes
import os

from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
logger = logging.getLogger(__name__)

# import opf class created by jwang
from opf import dcopf

## Test DCOPF model created by Jinning Wang


## Test (main)

show examples of editing dataframe


In [14]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_vsg.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

In [15]:
stg = build_group_table(ssa, 'StaticGen', ['u', 'idx']).reset_index(drop=True)

In [16]:
stg
# build_group_table    把选中两个andes file按照head 合并
# 'StaticGen' 在andes 里面默认包含 PV 和 slack
# 这种包含关系在 andes model reference 里面查

,u,idx
0,1.0,PV_2
1,1.0,PV_3
2,1.0,PV_4
3,1.0,PV_5
4,1.0,PV_6
5,1.0,PV_7
6,1.0,Slack_1


In [17]:
genrou = ssa.GENROU.as_df()[['idx', 'gen', 'M', 'D']].rename(columns={'idx':'genr'})

In [18]:
res = pd.merge(left=stg.rename(columns={'idx':'gen'}), right=genrou, on='gen', how='left')

In [19]:
res

,u,gen,genr,M,D
0,1.0,PV_2,GENROU_2,8.0,0.0
1,1.0,PV_3,GENROU_3,5.0,0.0
2,1.0,PV_4,GENROU_4,5.0,0.0
3,1.0,PV_5,GENROU_5,10.0,0.0
4,1.0,PV_6,NaN,NaN,NaN
5,1.0,PV_7,NaN,NaN,NaN
6,1.0,Slack_1,GENROU_1,8.0,0.0


In [20]:
vsg = ssa.REGCV1.as_df()[['idx', 'gen', 'M', 'D']].rename(columns={'idx':'vsg', 'M':'Mv', 'D':'Dv'})

In [21]:
res = pd.merge(left=res, right=vsg, on='gen', how='left')

In [22]:
res

,u,gen,genr,M,D,vsg,Mv,Dv
0,1.0,PV_2,GENROU_2,8.0,0.0,NaN,NaN,NaN
1,1.0,PV_3,GENROU_3,5.0,0.0,NaN,NaN,NaN
2,1.0,PV_4,GENROU_4,5.0,0.0,NaN,NaN,NaN
3,1.0,PV_5,GENROU_5,10.0,0.0,NaN,NaN,NaN
4,1.0,PV_6,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,PV_7,NaN,NaN,NaN,1.0,3.0,0.5
6,1.0,Slack_1,GENROU_1,8.0,0.0,NaN,NaN,NaN


In [ ]:
res = pd.merge(left=stg.rename(columns={'idx':'gen'}), right=vsg, on='gen', how='left')

In [ ]:
vsg.columns

Index(['idx', 'u', 'name', 'bus', 'gen', 'coi', 'coi2', 'Sn', 'Vn', 'fn', 'D',
       'M', 'ra', 'xl', 'xd1', 'kp', 'kw', 'S10', 'S12', 'gammap', 'gammaq',
       'xd', 'xq', 'xd2', 'xq1', 'xq2', 'Td10', 'Td20', 'Tq10', 'Tq20'],
      dtype='object')

In [ ]:
res[['gen', 'M', 'D']]

,gen,M,D
0,PV_2,8.0,0.0
1,PV_3,5.0,0.0
2,PV_4,5.0,0.0
3,PV_5,10.0,0.0
4,PV_6,NaN,NaN


on is key name

In [ ]:
build_group_table(ssa, 'StaticGen', [''])

TypeError: build_group_table() missing 1 required positional argument: 'columns'

In [ ]:
ss = dcopf()

NameError: name 'dcopf' is not defined

In [ ]:
ss.from_andes(ssa)

In [ ]:
ss.build()

Successfully build DCOPF model.


In [ ]:
ss.get_res()

Successfully build DCOPF model.


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 66 rows, 12 columns and 268 nonzeros
Model fingerprint: 0xd04bc91f
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 3e+00]
  RHS range        [1e-01, 4e+00]
Presolve removed 64 rows and 7 columns
Presolve time: 0.01s
Presolved: 2 rows, 5 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2370000e+00   0.000000e+00   0.000000e+00      0s
       0    2.2370000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.237000000e+00


Successfully solve DCOPF.


,gen,pg
0,PV_2,0.100
1,PV_3,0.100
2,PV_4,0.100
3,PV_5,0.100
4,PV_6,-0.100
5,Slack_1,1.937


## Test Attributes and Var

In [ ]:
ss.cost

,idx,c2,c1,c0,cr
0,PV_2,0,1,0,0
1,PV_3,0,1,0,0
2,PV_4,0,1,0,0
3,PV_5,0,1,0,0
4,PV_6,0,1,0,0
5,Slack_1,0,1,0,0


In [ ]:
ss.costdict

{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0}}

In [ ]:
ss.cost['c2'] = 2

In [ ]:
ss.cost

,idx,c2,c1,c0,cr
0,PV_2,2,1,0,0
1,PV_3,2,1,0,0
2,PV_4,2,1,0,0
3,PV_5,2,1,0,0
4,PV_6,2,1,0,0
5,Slack_1,2,1,0,0


In [ ]:
ss.gsf_matrix.shape

(16, 14)

In [ ]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,ctrl,ramp5,ramp10,ramp30
0,PV_2,1.0,PV_2,100.0,69.0,2,0.4,0.5,0.1,1.03,1,100,200,600
1,PV_3,1.0,PV_3,100.0,69.0,3,0.4,0.5,0.1,1.01,1,100,200,600
2,PV_4,1.0,PV_4,100.0,138.0,6,0.3,1.0,0.1,1.03,1,100,200,600
3,PV_5,1.0,PV_5,100.0,69.0,8,0.3,0.5,0.1,1.03,1,100,200,600
4,PV_6,1.0,PV_6,100.0,69.0,4,-0.01,0.1,-0.1,1.01,1,100,200,600
5,Slack_1,1.0,Slack_1,100.0,69.0,1,0.81442,3.0,0.5,1.03,1,100,200,600


In [ ]:
ss.pg

{'PV_2': <gurobi.Var pg[PV_2] (value 0.1)>,
 'PV_3': <gurobi.Var pg[PV_3] (value 0.1)>,
 'PV_4': <gurobi.Var pg[PV_4] (value 0.1)>,
 'PV_5': <gurobi.Var pg[PV_5] (value 0.1)>,
 'PV_6': <gurobi.Var pg[PV_6] (value -0.1)>,
 'Slack_1': <gurobi.Var pg[Slack_1] (value 1.9369999999999998)>}

In [ ]:
ss.gendict.keys()

dict_keys(['PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'Slack_1'])

In [ ]:
ss.costdict


{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0}}

In [ ]:
ss.load.p0.sum()

2.237

In [ ]:
ss.res_cost

2.237

In [ ]:
ss.mdl.display()

Minimize
  <gurobi.LinExpr: pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6] + pg[Slack_1]>
Subject To
PowerBalance: <gurobi.LinExpr: pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6] +
 pg[Slack_1]> = 2.237
Line_1_U: <gurobi.LinExpr: -0.8380258373889505 pg[PV_3] + -0.6105580062323537 pg[PV_4]
 + -0.6573114828670689 pg[PV_5] + -0.746539270176955 pg[PV_6]> <= 0.4592
Line_1_D: <gurobi.LinExpr: -0.8380258373889505 pg[PV_3] + -0.6105580062323537 pg[PV_4]
 + -0.6573114828670689 pg[PV_5] + -0.746539270176955 pg[PV_6]> >= -3.5408
Line_2_U: <gurobi.LinExpr: -0.16197416261104897 pg[PV_3] + -0.3894419937676453 pg[PV_4]
 + -0.3426885171329301 pg[PV_5] + -0.25346072982304424 pg[PV_6]> <= 0.3038
Line_2_D: <gurobi.LinExpr: -0.16197416261104897 pg[PV_3] + -0.3894419937676453 pg[PV_4]
 + -0.3426885171329301 pg[PV_5] + -0.25346072982304424 pg[PV_6]> >= -1.6962
Line_3_U: <gurobi.LinExpr: 0.0273438408947506 pg[PV_3] + -0.10307200056050626 pg[PV_4]
 + -0.1427239899061679 pg[PV_5] + -0.5320311371153

In [ ]:
ss.gen.idx

0       PV_2
1       PV_3
2       PV_4
3       PV_5
4       PV_6
5    Slack_1
Name: idx, dtype: object

In [ ]:
ss.gen.idx[0]

'PV_2'

In [ ]:
ss.pg

{'PV_2': <gurobi.Var pg[PV_2] (value 0.1)>,
 'PV_3': <gurobi.Var pg[PV_3] (value 0.1)>,
 'PV_4': <gurobi.Var pg[PV_4] (value 0.1)>,
 'PV_5': <gurobi.Var pg[PV_5] (value 0.1)>,
 'PV_6': <gurobi.Var pg[PV_6] (value -0.1)>,
 'Slack_1': <gurobi.Var pg[Slack_1] (value 1.9369999999999998)>}

In [ ]:
ss.pg['PV_2'].X

0.1